In [ ]:
#WebScraping do Kabum, produtos mais novos no site

import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import math

# Website a fazer scraping
website = 'https://www.kabum.com.br/acabaramdechegar'

# headers para não dar ruim no site ao raspar
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
}
site = requests.get(website, headers=headers)
soup = BeautifulSoup(site.content, 'html.parser')

# Fazer o calculo pra fazer a quantidade de páginas total
qtd_itens = soup.find('div', id='listingCount').get_text().strip()

# Limpar o valor
index = qtd_itens.find(' ')
qtd = qtd_itens[:index]

# Verificar ultima pagina, 'ceil' para quebrar pra cima a numeração
ultima_pagina = math.ceil(int(qtd)/20)

# Dicionario para armazenar os dados
dict_produtos = {'descricao':[], 'preco':[], 'frete_gratis':[]}

# Fazendo a raspagem (todas as páginas é muita página)
for i in range(1, ultima_pagina - 450 ):
    
    url_page = f'https://www.kabum.com.br/acabaramdechegar?page_number={i}&page_size=20&facet_filters=&sort=-date_product_arrived'
    site = requests.get(url_page, headers=headers)
    soup = BeautifulSoup(site.content, 'html.parser')
    
    # Criando uma variável para armazenar os cards principais, organizar melhor pro for sequente 
    produtos = soup.find_all('div', class_=re.compile('productCard'))
    
    for produto in produtos:
        
        # try pra caso não tiver a info, não dar erro no código
        try:
            descricao = produto.find('span', class_=re.compile('nameCard')).get_text().strip()
        except:
            descricao = 'n/a'
       
        try:
            preco = produto.find('span', class_=re.compile('priceCard')).get_text().strip()
        except:
            preco = 'n/a'
            
        try:
            frete_gratis = produto.find('div', class_=re.compile('freeShippingTagCard')).get_text().strip()
        except:
            frete_gratis = 'n/a'
            
        print(descricao, preco, frete_gratis)
        
        # Armazenando no dicionário os dados
        dict_produtos['descricao'].append(descricao)
        dict_produtos['preco'].append(preco)
        dict_produtos['frete_gratis'].append(frete_gratis)

# Criando um dataframe com o dicionário e mandando pra um arquivo .csv    
df = pd.DataFrame(dict_produtos)
df.to_csv('C:/Users/Danilo/Documents/repos/novidades_kabum.csv', encoding='utf-8', sep=';')

        
    